**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests. 

[Datasets](https://drive.google.com/drive/folders/1D6D-zRU3oiP12c0dFMQjs66JDhBZBtax?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1JfwvumxS2oys8ZcQtsb8GGmeRfnkzFdy/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1W7x5_PU5KszT8mqXqqEmRfXuR8CRr1DX/view?usp=sharing)

# 1. (2 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate. 

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [1]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from math import *
from scipy import stats

from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
# set the path to the datasets for A5
fpath = 'drive/MyDrive/A5/'
os.listdir(fpath)

['facebook_ads.csv', 'starbucks_promos.csv']

In [3]:
# read in the facebook_ads.csv file as the dataframe variable named fb
fb = pd.read_csv('drive/MyDrive/A5/facebook_ads.csv')
fb.head()



,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [4]:
# Take a look at the first 5 rows
# Observe that each row represents 1 ad from 1 campaign and a target demographic group (age, gender, interest)
fb.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [5]:
# use a groupby to create new dataframe, sumstats that tabulates the total clicks and impressions for each campaign.
# Generic groupby syntax: sumstats = df.groupby('groupbyvariable')[['summary_variable1', 'summary_variable2']].summaryfunction().reset_index()
# use the correct dataframe name, variables (i.e. column names) and summaryfunction

sumstats = fb.groupby('xyz_campaign_id')[['Impressions', 'Clicks']].sum().reset_index()
sumstats.head()

,xyz_campaign_id,Impressions,Clicks
0,916,482925,113
1,936,8128187,1984
2,1178,204823716,36068


In [6]:
# create a new column in sumstats called 'ctr' (short for click through rate)
# recall that ctr = clicks / impressions
sumstats["ctr"] = (sumstats["Impressions"] / sumstats["Clicks"]) 



In [7]:
# take a look at the whole sumstats dataframe
sumstats.head()


,xyz_campaign_id,Impressions,Clicks,ctr
0,916,482925,113,4273.672566
1,936,8128187,1984,4096.868448
2,1178,204823716,36068,5678.821005


In [8]:
# create a new column ctr_sd for the std. dev of the ctr (See hint above)
# pay attention to PEMDAS, esp. that you want to take the sqrt after computing p*(1-p)
# np.sqrt(x) returns the sqrt of some number x
# alternatively, x**.5 also returns the sqrt of x

ctr_std = sumstats.ctr.std(ddof=0)
ctr_std








707.756684771233

In [9]:
# take another look at the whole sumstats dataframe
# compare the ctr_std to calculating it by hand, make sure you did this correctly.
sumstats



,xyz_campaign_id,Impressions,Clicks,ctr
0,916,482925,113,4273.672566
1,936,8128187,1984,4096.868448
2,1178,204823716,36068,5678.821005


## Answer (edit this cell)
The campaign with the highest CTR is: **__Campaign 1178_____________**


# 2. (4 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget? 

In [10]:
# use stats.norm.ppf([.025, .975], mean, std error)
# where mean is sample mean and std error is std dev / sqrt(obs) (from Central Limit Theorem)
numberResults = fb.pivot_table(columns=['xyz_campaign_id'], aggfunc='size')
responses916= numberResults.get(916)
ctr916 = sumstats['ctr'].loc[0]
numbObs = len(fb)

# confidence interval for ad 916
stats.norm.ppf([.5,.95], ctr916/responses916, ctr_std/sqrt(numbObs))


array([ 79.14208456, 113.57613582])

In [11]:
# confidence interval for ad 936

responses936= numberResults.get(936)
ctr936 = sumstats['ctr'].loc[1]
numbObs = len(fb)

stats.norm.ppf([.5,.95], ctr936/responses936, ctr_std/sqrt(numbObs))

array([ 8.82945786, 43.26350912])

In [12]:
# confidence interval for ad 1178

responses1178= numberResults.get(1178)
ctr1178 = sumstats['ctr'].loc[2]
numbObs = len(fb)

stats.norm.ppf([.5,.95], ctr1178/responses1178, ctr_std/sqrt(numbObs))



array([ 9.08611361, 43.52016486])

**Edit this cell**

1. (write 1-2 sentences to compare the confidence interval of 916 vs 936's CTR)
**We are 95% confident that the result of 916's CTR will fall in the interval of (79.14208456, 113.57613582), and we are 95% confident that the result of 936's CTR will fall in the interval of (8.82945786, 43.26350912). Therefore on average the CTR of 916 is much higher than 936.** 

1. (write 1-2 sentences to compare the confidence interval of 916 vs 1178's CTR)
**We are 95% confident that the result of 916's CTR will fall in the interval of (79.14208456, 113.57613582), and we are 95% confident that the result of 1178's CTR will fall in the interval of (9.08611361, 43.52016486). Therefore on average the CTR of 916 is much higher than 1178.** 

In [14]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):
    """
    This function requires you tu supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2) 
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))
    
    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [16]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 916

ttest_2sample((ctr936/responses936), ctr_std, numbObs, (ctr916/responses916), ctr_std, numbObs )



0.017632692729846532

In [19]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 1178
ttest_2sample((ctr936/responses936), ctr_std, numbObs, (ctr1178/responses1178), ctr_std, numbObs )

0.9930838848959749

**Edit this cell to answer**

(Write a few sentences to describe how to proceed with the allocation of the ad budget going forward. Be sure to use the 2 tail t-tests above to support your strategy) **The p-value in the comparison of 936 and 916 is .017 which is less than .05, which means that we reject the null and that there is a difference in the population means. The p-value in the comparison of 936 and 1178 is .993 which is greater than .05 so we accept the null meaning the population means are the same. **
**The alloaction of the ad budget would have to be adjusted accordingly because this means that 936 and 1178 have a much larger population, meaning even if their CTR is lower they should recieve more budgeting**

# 3. (4 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [21]:
# read in the starbucks_promos.csv file as the dataframe variable named sb, use index_col = 0 option
sb = pd.read_csv('drive/MyDrive/A5/starbucks_promos.csv', index_col = 0)


In [23]:
# preview first 5 rows to get a sense of the contents of the dataframe
# note that each observation represents an offer received by a customer
sb.head(5)

,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [44]:
# select the reemed column for discount offers, name this data the variable disc
disc= sb.loc[sb["offer_type"]=="discount"]
disc["redeemed"]

1        1
8        1
13       1
17       1
19       0
        ..
76259    1
76264    1
76266    1
76272    0
76276    1
Name: redeemed, Length: 30543, dtype: int64

In [40]:
# select the reemed column for buy one get one (bogo) offers, name this data the variable bogo
bogo= sb.loc[sb["offer_type"]=="bogo"]
bogo.head()


,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
9,00857b24b13f4fe0ad17b605f00357f5,offer received,0,M,71,20171023,41000.0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0,"['web', 'email', 'mobile', 'social']",10.0,120.0,bogo,0.0,NaN,NaN,0
10,008d7088107b468893889da0ede0df5c,offer received,0,M,24,20170910,42000.0,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"['web', 'email', 'mobile', 'social']",5.0,120.0,bogo,0.0,NaN,NaN,0


In [65]:
# use stats.ttest_ind to test whether the 2 types of offers yield different redemption rates

stats.ttest_ind(bogo["redeemed"],disc["redeemed"])


Ttest_indResult(statistic=-17.30160421286854, pvalue=6.611390792545181e-67)

In [88]:
# Select the income column for those who redeemed the BOGO offer, call this the variable bogo_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for bogo offer, and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
bogo_income=bogo["income"].loc[bogo["redeemed"]==1]
bogo_income=bogo_income.dropna()
bogo_income


5         91000.0
6        119000.0
11        81000.0
23        80000.0
24       102000.0
           ...   
76263     92000.0
76268     77000.0
76271     93000.0
76273     73000.0
76275     34000.0
Name: income, Length: 15933, dtype: float64

In [87]:
# Select the income column for those who redeemed the discount offer, call this the variable disc_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for discount offer,  and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
disc_income=disc["income"].loc[disc["redeemed"]==1]
disc_income=disc_income.dropna()
disc_income



1         90000.0
8         61000.0
13        66000.0
17       102000.0
21        72000.0
           ...   
76256     53000.0
76259     86000.0
76264     32000.0
76266     47000.0
76276     62000.0
Name: income, Length: 17710, dtype: float64

In [53]:
# What is the average income for bogo redeemers (bogo_income.mean())?
bogo_income.mean()


70114.8559593297

In [54]:
# What is the average income for discount redeemers?
disc_income.mean()


68617.39130434782

In [89]:
# use stats.ttest_ind to test whether the average income of the redeemers of the 2 offer types are different
stats.ttest_ind(bogo_income, disc_income)


Ttest_indResult(statistic=6.343377852366808, pvalue=2.2763463492984577e-10)

**Edit this cell**
1. (Write a sentence to describe the relative redemption rates of the 2 offer types based on the result of the t-test).
**Based on the t-test we reject the null hypothesis and the relative redemption rates are not statistically significant**
1. (Write a sentence to describe the relative incomes of the redeemers of the 2 offer types based on the result of the t-test).
**Based on the t-test we reject the null hypothesis and the income rates are not statistically significant**